# Extra material: Shortest path in real life


In [1]:
import subprocess
import sys

class ColabInstaller():

    def __init__(self):
        reqs = subprocess.check_output([sys.executable, '-m', 'pip', 'freeze'])
        self.installed_packages = [r.decode().split('==')[0] for r in reqs.split()]

    def on_colab(self):
        return "google.colab" in sys.modules

    def install(self, package):
        if self.on_colab():
            subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", package])
            self.installed_packages.append(package)

    def upgrade(self, package):
        if self.on_colab():
            subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "--upgrade", package])

colab = ColabInstaller()
colab.upgrade('numpy')
colab.upgrade('matplotlib')
colab.install('geopandas')
colab.install('geopy')
colab.install('osmnx')
colab.install('osmnet')
colab.install('pandana')
colab.upgrade('geopy')

# Introduction

Google brought with [maps]( https://www.google.com/maps) the world to our screens, including accurate geocoding and routing for several modalities. For the most, the usage of [maps]( https://www.google.com/maps) is interactive. As data and analytics professionals we often need a programmatically support for the services that [maps]( https://www.google.com/maps) offer us, preferably free. It also offers a plethora of [development support](https://developers.google.com/), but unfortunately most is paid. That is even more so for [maps]( https://developers.google.com/maps/documentation).

## Some background information and history 
[Geoff Boeing]( https://geoffboeing.com/about/) is a true leader in demystifying urban data analytics, with a strong emphasis on street networks. His [peer reviewed publications]( https://geoffboeing.com/publications/) are open and accompanied by usable demonstrations using his own [OSMnx]( https://geoffboeing.com/2018/03/osmnx-features-roundup/) package.
Professor [Peter Sanders]( https://algo2.iti.kit.edu/english/sanders.php), see also his [Wikipedia]( https://en.wikipedia.org/wiki/Peter_Sanders_(computer_scientist)) page, has moved his interests to other areas but his [route planning]( http://algo2.iti.kit.edu/routeplanning.php) project shaped the world of truly scalable road routing algorithms. 
From his alumni I distinguish two persons:
 * [Dominik Schultes](http://algo2.iti.kit.edu/schultes/) who won the [DIMACS challenge on shortest paths]( http://www.diag.uniroma1.it//challenge9/data/tiger/) and made it to the [Scientific American top 50]( https://www.scientificamerican.com/article/sciam-50-the-fastest-way/). Before Dominik’s research scalable shortest paths on large national road networks where heuristics, now they are exact and can be computed at world scale. 
 * [Dennis Luxen]( http://algo2.iti.kit.edu/english/luxen.php) for creating https://github.com/Project-OSRM/osrm-backend which offers a free, scalable, implementation of [contraction hierarchies]( https://en.wikipedia.org/wiki/Contraction_hierarchies).
 
Finally, I mention [Fletcher Foti]( https://fletcherfoti.weebly.com/) who gave us [pandana]( http://udst.github.io/pandana/).
 


## Geocoding

The world is mapped with the [geographic coordinate system](https://en.wikipedia.org/wiki/Geographic_coordinate_system) but we have difficulties remembering [latitudes]( https://en.wikipedia.org/wiki/Latitude) and [longitudes]( https://en.wikipedia.org/wiki/Longitude).

We learn and remember the world better from addresses. 


In [2]:
import geopy
    
def FreeLocator():
    return geopy.Photon(user_agent='myGeocoder')

Amsterdam = FreeLocator().geocode('Amsterdam,NL')

# Visualization

In [3]:
import folium
Map = folium.Map(location=(Amsterdam.latitude,Amsterdam.longitude), zoom_start=12)
Map

In [5]:
def locate_geopy(description):
    location = FreeLocator().geocode(description)
    if location is not None:
        return location.latitude, location.longitude
    return None, None

In [6]:
import pandas as pd
pd.options.display.float_format = '{:.6f}'.format

data = {'address': [ 'Centraal Station',
                     'Amsterdam Business School',
                     'Artis',
                     'Arena',
                     'Ziggo Dome' ], 
        'color'  : [ 'blue',
                     'black',                   
                     'green',
                     'red',
                     'purple' ]}
# Create DataFrame.
df = pd.DataFrame(data)
df['city']    = 'Amsterdam'
df['country'] = 'NL'
df

,address,color,city,country
0,Centraal Station,blue,Amsterdam,NL
1,Amsterdam Business School,black,Amsterdam,NL
2,Artis,green,Amsterdam,NL
3,Arena,red,Amsterdam,NL
4,Ziggo Dome,purple,Amsterdam,NL


In [7]:
locations = [ locate_geopy(','.join(row[['address','city','country']])) for _, row in df.iterrows() ]
df['lat'] = [ loc[0] for loc in locations ]
df['lon'] = [ loc[1] for loc in locations ]
df

,address,color,city,country,lat,lon
0,Centraal Station,blue,Amsterdam,NL,52.378901,4.900581
1,Amsterdam Business School,black,Amsterdam,NL,52.365107,4.911718
2,Artis,green,Amsterdam,NL,-30.401207,-56.481521
3,Arena,red,Amsterdam,NL,52.315990,4.942931
4,Ziggo Dome,purple,Amsterdam,NL,52.313629,4.938207


In [8]:
for _, row in df.iterrows():
    folium.Marker((row.lat,row.lon),icon=folium.Icon(color=row.color), tooltip=row.address).add_to(Map)
Map

In [9]:
import osmnx as ox
import networkx as nx
from IPython.display import display

ox.config(log_console=True, use_cache=True)

/Users/ale/.pyenv/versions/3.10.6/lib/python3.10/site-packages/osmnx/utils.py:192: UserWarning: The `utils.config` function is deprecated and will be removed in a future release. Instead, use the `settings` module directly to configure a global setting's value. For example, `ox.settings.log_console=True`.
  warnings.warn(


In [10]:
%%time 
G_walk = ox.graph_from_place('Amsterdam, NL', network_type='walk')

CPU times: user 58 s, sys: 1.07 s, total: 59.1 s
Wall time: 2min 36s


In [11]:
print( G_walk.number_of_nodes(), G_walk.number_of_edges() )

37079 104568


In [12]:
df['osmnx'] = ox.distance.nearest_nodes(G_walk,df.lon,df.lat)
df

,address,color,city,country,lat,lon,osmnx
0,Centraal Station,blue,Amsterdam,NL,52.378901,4.900581,5629072001
1,Amsterdam Business School,black,Amsterdam,NL,52.365107,4.911718,46356661
2,Artis,green,Amsterdam,NL,-30.401207,-56.481521,1768194163
3,Arena,red,Amsterdam,NL,52.315990,4.942931,4622542635
4,Ziggo Dome,purple,Amsterdam,NL,52.313629,4.938207,1925143759


In [13]:
%time route = nx.shortest_path(G_walk,df.iloc[0].osmnx,df.iloc[1].osmnx,weight='length')
print(route)

CPU times: user 24.5 ms, sys: 2.73 ms, total: 27.2 ms
Wall time: 26.8 ms
[5629072001, 5629072000, 5629071975, 4239313191, 4239313081, 4239313075, 4239312638, 4239312409, 9683540350, 4239311933, 4239310797, 3189915006, 3175727789, 1263352467, 9807404628, 9959600367, 9283437392, 9982135036, 9982135039, 9982135032, 9982135049, 9982118415, 9971788071, 9807403578, 46411146, 46410382, 8238495242, 46409003, 46405684, 46402187, 5933448325, 4959039686, 9407636386, 1291587477, 9913510572, 8223730972, 1291587576, 8223730971, 8223752620, 46383339, 46380741, 46379359, 46374887, 46374082, 3781170573, 7995426344, 3781170150, 3781170147, 3781170139, 2021897089, 2021897044, 7223158294, 7223158307, 2021897032, 7223210075, 7223210071, 1291468055, 3160068713, 3160068154, 46356661]


In [14]:
route_map = ox.plot_route_folium(G_walk, route)
display(route_map)

In [15]:
%%time 
nodes = pd.DataFrame.from_dict(dict(G_walk.nodes(data=True)), orient='index')

CPU times: user 118 ms, sys: 10.3 ms, total: 128 ms
Wall time: 131 ms


In [16]:
%%time 
edges = nx.to_pandas_edgelist(G_walk)

/Users/ale/.pyenv/versions/3.10.6/lib/python3.10/site-packages/pandas/core/dtypes/cast.py:127: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)


CPU times: user 4.23 s, sys: 66.7 ms, total: 4.29 s
Wall time: 4.32 s


In [17]:
nodes.street_count.describe()

count   37079.000000
mean        2.821840
std         0.994714
min         1.000000
25%         3.000000
50%         3.000000
75%         3.000000
max         7.000000
Name: street_count, dtype: float64

In [18]:
edges.length.describe()

count   104568.000000
mean        52.067069
std         66.243924
min          0.048000
25%         14.511750
50%         32.674000
75%         65.676500
max       1550.813000
Name: length, dtype: float64

In [19]:
edges.loc[edges.length.idxmax()]

source                                              2632720004
target                                              4340833704
osmid                                                  7039463
length                                             1550.813000
width                                                      NaN
junction                                                   NaN
tunnel                                                     NaN
lanes                                                      NaN
oneway                                                   False
geometry     LINESTRING (5.0484437 52.4098672, 5.0492122 52...
est_width                                                  NaN
name                                             Uitdammerdijk
reversed                                                 False
highway                                               tertiary
service                                                    NaN
ref                                                    

In [20]:
%time longest = nx.shortest_path(G_walk,2632720004,46544942,weight='length')
print(longest)

CPU times: user 815 µs, sys: 1e+03 ns, total: 816 µs
Wall time: 826 µs
[2632720004, 4868500506, 46526747, 3646762450, 46515220, 7896482872, 7896482772, 7896482536, 2876000370, 2876030674, 46525998, 2875982072, 2876049179, 2876046111, 2834850861, 3380450298, 3380450325, 3380450323, 1970783754, 1970783822, 1970783783, 1970783792, 1970789262, 1970785372, 46546263, 3478950226, 2869259996, 2876043898, 46544942]


In [21]:
route_map = ox.plot_route_folium(G_walk, longest)
display(route_map)

# Dijkstra on steroids for road networks

In [22]:
%%time
import pandana
network = pandana.Network(nodes['x'], nodes['y'], edges['source'], edges['target'], edges[['length']],twoway=True)

ModuleNotFoundError: No module named 'pandana'

In [23]:
network.nodes_df.head()

NameError: name 'network' is not defined

In [ ]:
network.edges_df.head()

In [ ]:
df['pandana'] = network.get_node_ids(df.lon, df.lat).values
df

In [ ]:
%time path_pandana = network.shortest_path(df.iloc[2].pandana, df.iloc[3].pandana)

In [ ]:
%time path_nx = nx.shortest_path(G_walk,df.iloc[2].osmnx,df.iloc[3].osmnx,weight='length')

In [ ]:
A = set(path_pandana)
B = set(path_nx)
(A | B) - (A & B)

In [ ]:
origs = [o for o in df.pandana for d in df.pandana]
dests = [d for o in df.pandana for d in df.pandana]
%time distances = network.shortest_path_lengths(origs, dests)

In [ ]:
import numpy as np 

n = len(df)
pd.options.display.float_format = '{:.2f}'.format
pd.DataFrame(np.array(list(distances)).reshape(n,n),index=df.address,columns=df.address)

In [ ]:
np.random.seed(2021)
n = 500
sample = np.random.choice(np.array(network.nodes_df.index.values.tolist()), n, replace=False)
origs = [o for o in sample for d in sample]
dests = [d for o in sample for d in sample]

In [ ]:
%time distances = network.shortest_path_lengths(origs, dests)
%time table = pd.DataFrame(np.array(list(distances)).reshape(n,n),index=sample,columns=sample)

In [ ]:
departure = table.max(axis=1).idxmax()
arrival = table.loc[departure].idxmax()
%time path_pandana = network.shortest_path(departure, arrival)
%time path_nx = nx.shortest_path(G_walk,departure,arrival,weight='length')
A = set(path_pandana)
B = set(path_nx)
(A | B) - (A & B)

In [ ]:
%time paths = network.shortest_paths(origs,dests)

In [ ]:
sum(map(len,paths))

In [ ]:
for u,v in zip(paths[1][:-1],paths[1][1:]):
    print(G_walk.get_edge_data(u,v)[0].get('name',''))

In [ ]:
route_map = ox.plot_route_folium(G_walk, paths[1],color='red',map=Map)
display(route_map)